# Synthesize Data With FDKT

In this tutoria, we will demonstrate how to Synthesize data using the FATE-LLM framework. In FATE-LLM, we introduce the "FDKT" module,  specifically designed for domain-specific knowledge transfer on large language models using synthetic data. FDKT Algorithm is based on paper [Federated Domain-Specific Knowledge Transfer on
Large Language Models Using Synthetic Data](https://arxiv.org/pdf/2405.14212), We integrate its code into the FATE-LLM framework.  


## Dataset: Yelp
We processed and sample data of 'Health' subdomain from [Yelp dataset](https://arxiv.org/abs/1509.01626) , the dataset can be downloaded from [here](https://www.yelp.com/dataset). 
Once the dataset has been downloaded, execute the following command to untar the downloaded dataset.

In [ ]:
```shell
tar -xvf yelp_dataset.tar
```

The following code will sample 5000 datalines of 'Health' subdomain, and train data will generated under the folder './balance_processed_data/Health/train.json'

In [12]:
import os
import json
import sys
import random
from pathlib import Path
random.seed(42)


base_dir = "./"
business_data_path = os.path.join(base_dir, 'yelp_academic_dataset_business.json')
review_data_path = os.path.join(base_dir, 'yelp_academic_dataset_review.json')

business_data_file = open(business_data_path, 'r')
review_data_file = open(review_data_path, 'r')

categories_list = ['Restaurants', 'Shopping', 'Arts', 'Health']
business_dic = {}
data_dict = {}
for category in categories_list:
    business_dic[category] = set()
    data_dict[category] = []


def get_categories(categories):
    return_list = []
    for category in categories_list:
        if category in categories:
            return_list.append(category)
    return return_list


for line in business_data_file.readlines():
    dic = json.loads(line)
    if 'categories' in dic.keys() and dic['categories'] is not None:
        category = get_categories(dic['categories'])
        if len(category) == 1:
            business_dic[category[0]].add(dic['business_id'])

# for category in categories_list:
for line in review_data_file.readlines():
    dic = json.loads(line)
    if 'business_id' in dic.keys() and dic['business_id'] is not None:
        for category in categories_list:
            if dic['business_id'] in business_dic[category]:
                if dic['text'] is not None and dic['stars'] is not None:
                    data_dict[category].append({'text': dic['text'], 'stars': dic['stars']})
                break

train_data_path = os.path.join('processed_data', "Health", 'train.json')
os.makedirs(Path(train_data_path).parent, exist_ok=True)
train_data_file = open(train_data_path, 'w')
data_list = data_dict["Health"]

sample_data_dict = dict()

for data in data_list:
    star = int(data["stars"])
    if star not in sample_data_dict:
        sample_data_dict[star] = []

    sample_data_dict[star].append(data)

data_list = []
star_keys = list(sample_data_dict.keys())
for star in star_keys:
    sample_data = sample_data_dict[star][:1000]
    random.shuffle(sample_data)
    data_list.extend(sample_data)

random.shuffle(data_list)
json.dump(data_list, train_data_file, indent=4)
train_data_file.close()



## Models Use
Please download the following models, these models are used for data augmentation process.

LLM: [Qwen1.5-7B-Chat](https://huggingface.co/Qwen/Qwen1.5-7B-Chat)  
SLM: [gpt2-xl](https://huggingface.co/openai-community/gpt2-xl)

MeanWhile, 'all-mpnet-base-v2' is used to generate embedding vectors in LLM side.

Embedding Model:  [all-mpnet-base-v2](https://huggingface.co/sentence-transformers/all-mpnet-base-v2)


## Running FDKT Data Synthetic Process With Launcher (Experimential Using)

### SLM Setting

In this section, we will introduce some key configurations in SLM side.

#### 1. loading model

In [ ]:
import torch
import transformers
from fate_llm.data.tokenizers.cust_tokenizer import get_tokenizer


slm_pretrained_path = "gpt2-xl" # modity this to local directory
slm = transformers.AutoModelForCausalLM.from_pretrained(slm_pretrained_path, torch_dtype=torch.bfloat16)
tokenizer = get_tokenizer(slm_pretrained_path)
tokenizer.pad_token_id = tokenizer.eos_token_id


#### 2. Initialize SLM Training Arugments

In [ ]:
from fate_llm.algo.fdkt.fdkt_data_aug import FDKTTrainingArguments


training_args = FDKTTrainingArguments(
    use_cpu=False, # use gpu to do dp(differential privacy) training process
    device_id=0, # the device number of gpu
    num_train_epochs=1, # dp training epochs
    per_device_train_batch_size=2, # batch size of dp training
    slm_generation_batch_size=32, # batch_size to generate data in slm side
    seq_num_for_single_category=300, # data num for each category(label)
    slm_generation_config=dict(
        max_new_tokens=256,
        temperature=1.0,
        top_k=50,
        top_p=0.9,
        repetition_penalty=1.0,
        pad_token_id=tokenizer.eos_token_id
    ),
)

#### 3. Initlaize DataSet Instance

We provide default templates for dataset "Yelp" and "AGNews", user can refer [here](https://github.com/FederatedAI/FATE-LLM/tree/dev-2.2.0/python/fate_llm/dataset/data_config) for more details. If you want to use your own dataset, please provide fields label_key/text_key/augment_format/filter_format/tokenize_format/sub_domain/label_list/few_shot_format/text_with_label_format like the two default templates and passing it as and argument.

In [ ]:
from fate_llm.dataset.flex_dataset import FlexDataset


ds = FlexDataset(
    tokenizer_name_or_path=slm_pretrained_path,
    load_from="json",
    data_part="train",
    dataset_name="yelp_review", # use default template
    # config=dict/template_path # if dataset_name not equals to "yelp_review" or "ag_news"
    need_preprocess=True,
    select_num=2000, # use data_num=2000 to train, default is None, None means using all data
)

### LLM Setting

In this section, we will introduce some key configurations in LLM side.

#### 1. Deploy VLLM Server And Use OpenAI API Protocol To SpeedUp LLM Inference

please copy the following code to local file create_and_start_vllm.sh, then run the bash code by executing "bash create_and_start_vllm.sh"

In [ ]:
# create_and_start_vllm.sh
# create vllm enviroment

python -m venv vllm_venv
source vllm_venv/bin/activate
pip install vllm==0.4.3
pip install numpy==1.26.4 # numpy >= 2.0.0 will raise error, so reinstall numpy<2.0.0

# please modify Qwen1.5-7B-Chat to local llm model saving path
export CUDA_VISIBLE_DEVICES=1,2
nohup python -m vllm.entrypoints.openai.api_server --host 127.0.0.1 --port 9999 --model Qwen1.5-7B-Chat --dtype=half --enforce-eager --api-key demo --device cuda -tp 2 &

#### 2. Initialize LLM Training Arugments

In [ ]:
from fate_llm.algo.fdkt.fdkt_data_aug import FDKTTrainingArguments


training_args = FDKTTrainingArguments(
    sample_num_per_cluster=4, # use this to estimate the number of clusters, n_clusters=(len(dataset) + sample_num_per_cluster - 1) // sample_num_per_cluster
    filter_prompt_max_length=2**16,
    filter_generation_config=dict(
        max_tokens=512,
    ),
    aug_generation_config=dict(
        max_tokens=4096,
        temperature=0.8,
        top_p=0.9,
    ),
    aug_prompt_num=20000, # prompts use for data augmentation
)

#### 3. Initialize Embedding Generated Model

In [ ]:
from fate_llm.model_zoo.embedding_transformer.st_model import SentenceTransformerModel


embedding_lm = SentenceTransformerModel(model_name_or_path="all-mpnet-base-v2").load() # modified model_name_or_path to local model saved path

#### 4. Initalize OpenAI Api For Inference

In [ ]:
from fate_llm.algo.fdkt.inference_inst import api_init


inference_inst = api_init(
    api_url="http://127.0.0.1:9999/v1/",
    model_name="Qwen1.5-7B-Chat", # modified model_name to local Meta-Llama-3-8B-Instruct saved path
    api_key="demo"
)

### Complete Code 

Please paste the code in "run_fdkt_by_launcher.py" and execute it with the following command. Once the process is finished, augmentation data will be saved in the current directory, whose filename is aug_data_result.json

In [ ]:
python run_fdkt_by_launcher.py --parties guest:9999 arbiter:10000 --log_level INFO

In [ ]:
import json
import os

import torch
from fate.arch import Context
from fate.arch.launchers.multiprocess_launcher import launch

# please replace the following four variables to local paths
llm_pretrained_path = "Qwen1.5-7B-Chat"
embedding_model_path = "all-mpnet-base-v2"
slm_pretrained_path = "gpt2-xl"
slm_data_path = "./process/Health/train.json"


def get_optimizer(model, optimizer="adam", lr=1e-4):
    if optimizer == "adam":
        optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
    elif optimizer == "adamw":
        optimizer = torch.optim.AdamW(params=model.parameters(), lr=lr)
    else:
        raise NotImplementedError("Given optimizer type is not supported")
    return optimizer


def train_slm(ctx):
    import transformers
    from fate_llm.algo.fdkt.fdkt_data_aug import (
        FDKTSLM,
        FDKTTrainingArguments
    )
    from fate_llm.dataset.flex_dataset import FlexDataset
    from fate_llm.data.tokenizers.cust_tokenizer import get_tokenizer
    from transformers.data import DataCollatorForSeq2Seq

    slm = transformers.AutoModelForCausalLM.from_pretrained(slm_pretrained_path, torch_dtype=torch.bfloat16)
    tokenizer = get_tokenizer(slm_pretrained_path)
    tokenizer.pad_token_id = tokenizer.eos_token_id
    training_args = FDKTTrainingArguments(
        use_cpu=False,
        device_id=0,
        num_train_epochs=1,
        per_device_train_batch_size=2,
        slm_generation_batch_size=32,
        seq_num_for_single_category=2000,
        slm_generation_config=dict(
            max_new_tokens=256,
            temperature=1.0,
            top_k=50,
            top_p=0.9,
            repetition_penalty=1.0,
            pad_token_id=tokenizer.eos_token_id
        ),
        # inference_method="vllm",
    )

    ds = FlexDataset(
        tokenizer_name_or_path=slm_pretrained_path,
        load_from="json",
        data_part="train",
        dataset_name="yelp_review",
        need_preprocess=True,
        select_num=2000,  # use 2000 data to train, default is None, using all data
    )
    ds.load(slm_data_path)

    fdkt_runner = FDKTSLM(
        ctx=ctx,
        model=slm,
        training_args=training_args,
        tokenizer=tokenizer,
        train_set=ds,
        optimizer=get_optimizer(slm),
        data_collator=DataCollatorForSeq2Seq(tokenizer, label_pad_token_id=tokenizer.pad_token_id)
    )

    aug_data = fdkt_runner.aug_data()
    with open("./aug_data_result.json", "w") as fout:
        fout.write(json.dumps(aug_data, indent=4))


def train_llm(ctx):
    from fate_llm.algo.fdkt.fdkt_data_aug import (
        FDKTLLM,
        FDKTTrainingArguments
    )
    from fate_llm.model_zoo.embedding_transformer.st_model import SentenceTransformerModel
    from fate_llm.dataset.flex_dataset import FlexDataset
    from fate_llm.algo.fdkt.inference_inst import api_init, vllm_init

    embedding_lm = SentenceTransformerModel(model_name_or_path=embedding_model_path).load()
    training_args = FDKTTrainingArguments(
        sample_num_per_cluster=5,
        filter_prompt_max_length=2**14,
        filter_generation_config=dict(
            max_tokens=4096,
        ),
        use_cpu=False,
        aug_generation_config=dict(
            max_tokens=4096,
            temperature=0.8,
            top_p=0.9,
        ),
        aug_prompt_num=20000,
    )

    ds = FlexDataset(
        tokenizer_name_or_path=llm_pretrained_path,
        load_from="json",
        data_part="train",
        dataset_name="yelp_review",
        need_preprocess=True,
        few_shot_num_per_label=1,
    )

    inference_inst = api_init(
        api_url="http://127.0.0.1:9999/v1/",
        model_name=llm_pretrained_path,
        api_key="demo"
    )

    fdkt_runner = FDKTLLM(
        ctx=ctx,
        embedding_model=embedding_lm,
        training_args=training_args,
        dataset=ds,
        inference_inst=inference_inst,
    )

    fdkt_runner.aug_data()


def run(ctx: Context):
    if ctx.is_on_arbiter:
        train_llm(ctx)
    else:
        os.environ["CUDA_VISIBLE_DEVICES"] = "0"
        train_slm(ctx)


if __name__ == "__main__":
    launch(run)

## Running FEDMKT with Pipeline (Industrial Using)

Please make sure that FATE and FATE-Flow has been deployed, paste the following code to test_fdkt_by_pipeline.py, the execute "python test_fdkt_by_pipeline.py"

In [ ]:
from fate_client.pipeline.components.fate.homo_nn import HomoNN, get_config_of_fdkt_runner
from fate_client.pipeline.components.fate.nn.algo_params import FDKTTrainingArguments
from fate_client.pipeline.components.fate.nn.loader import LLMModelLoader, LLMDatasetLoader, LLMDataFuncLoader
from fate_client.pipeline import FateFlowPipeline
from fate_client.pipeline.components.fate.reader import Reader
from fate_client.pipeline.components.fate.nn.torch import nn, optim


guest = '9999'# replace this party id to actual guest party id in your enviroment
arbiter = '9999'# replace this party id to actual arbiter party id in your enviroment

# please replace the following four variables to local paths
llm_pretrained_path = "Qwen1.5-7B-Chat"
embedding_model_path = "all-mpnet-base-v2/"
slm_pretrained_path = "gpt2-xl"
slm_data_path = "./process/Health/train.json" # should be absolute path


def get_llm_conf():
    embedding_model = LLMModelLoader(
        "embedding_transformer.st_model",
        "SentenceTransformerModel",
        model_name_or_path=embedding_model_path
    )

    dataset = LLMDatasetLoader(
        "flex_dataset",
        "FlexDataset",
        tokenizer_name_or_path=llm_pretrained_path,
        need_preprocess=True,
        dataset_name="yelp_review",
        data_part="train",
        load_from="json",
        few_shot_num_per_label=1,
    )

    training_args = FDKTTrainingArguments(
        sample_num_per_cluster=5,
        filter_prompt_max_length=2 ** 14,
        filter_generation_config=dict(
            max_tokens=4096,
        ),
        use_cpu=False,
        aug_generation_config=dict(
            max_tokens=4096,
            temperature=0.8,
            top_p=0.9,
        ),
        aug_prompt_num=20000,
    )

    inference_inst_conf = dict(
        module_name="fate_llm.algo.fdkt.inference_inst",
        item_name="api_init",
        kwargs=dict(
            api_url="http://127.0.0.1:9999/v1/",
            model_name=llm_pretrained_path,
            api_key="demo"
        )
    )

    return get_config_of_fdkt_runner(
        training_args=training_args,
        embedding_model=embedding_model,
        dataset=dataset,
        inference_inst_conf=inference_inst_conf,
    )


def get_slm_conf():
    slm_model = LLMModelLoader(
        "hf_model",
        "HFAutoModelForCausalLM",
        pretrained_model_name_or_path=slm_pretrained_path,
        torch_dtype="bfloat16",
    )

    tokenizer = LLMDataFuncLoader(
        "tokenizers.cust_tokenizer",
        "get_tokenizer",
        tokenizer_name_or_path=slm_pretrained_path,
        pad_token_id=50256
    )

    training_args = FDKTTrainingArguments(
        use_cpu=False,
        device_id=1,
        num_train_epochs=1,
        per_device_train_batch_size=2,
        slm_generation_batch_size=32,
        seq_num_for_single_category=2000,
        slm_generation_config=dict(
            max_new_tokens=256,
            temperature=1.0,
            top_k=50,
            top_p=0.9,
            repetition_penalty=1.0,
            pad_token_id=50256
        ),
    )

    dataset = LLMDatasetLoader(
        "flex_dataset",
        "FlexDataset",
        tokenizer_name_or_path=slm_pretrained_path,
        need_preprocess=True,
        dataset_name="yelp_review",
        data_part="train",
        load_from="json",
        select_num=2000,
        few_shot_num_per_label=1,
    )

    optimizer = optim.Adam(lr=0.01)

    return get_config_of_fdkt_runner(
        model=slm_model,
        tokenizer=tokenizer,
        training_args=training_args,
        dataset=dataset,
        optimizer=optimizer,
        data_collator=LLMDataFuncLoader(
            "data_collator.cust_data_collator",
            "get_seq2seq_data_collator",
            label_pad_token_id=50256,
            tokenizer_name_or_path=slm_pretrained_path,
            pad_token_id=50256,
        ),
    )


pipeline = FateFlowPipeline().set_parties(guest=guest, arbiter=arbiter)
pipeline.bind_local_path(path=slm_data_path, namespace="experiment", name="slm_train")


reader_0 = Reader("reader_0", runtime_parties=dict(guest=guest))
reader_0.guest.task_parameters(
    namespace="experiment",
    name="slm_train"
)


homo_nn_0 = HomoNN(
    'homo_nn_0',
    train_data=reader_0.outputs["output_data"],
    runner_module="fdkt_runner",
    runner_class="FDKTRunner",
)

homo_nn_0.arbiter.task_parameters(
    runner_conf=get_llm_conf()
)

homo_nn_0.guest.task_parameters(
    runner_conf=get_slm_conf()
)

pipeline.add_tasks([reader_0, homo_nn_0])
pipeline.conf.set("task", dict(engine_run={"cores": 1}))

pipeline.compile()
pipeline.fit()